In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 设置随机种子以保证可重复性
torch.manual_seed(42)

# 读取 csv_soh.csv
df_soh = pd.read_csv('csv_soc.csv', skiprows=1)  # 跳过第一行（表头）

# 提取列
labels = torch.tensor(df_soh.iloc[:, 0].values, dtype=torch.float32).view(-1, 1)  # 使用第一列作为标签
inputs = torch.tensor(df_soh.iloc[:, 1:4].values, dtype=torch.float32)  # 使用第二和第三列作为输入特征

# 将数据集拆分为训练集和测试集（80% 训练，20% 测试）
inputs_train, inputs_test, labels_train, labels_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)

# 转换为 PyTorch 数据集
train_dataset = TensorDataset(inputs_train, labels_train)
test_dataset = TensorDataset(inputs_test, labels_test)

# 创建训练集和测试集的 DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定义一个简单的神经网络模型
class SimpleDNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleDNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 实例化模型
input_size = 3  # 输入特征的数量
hidden_size = 8  # 隐藏层单元的数量
output_size = 1  # 输出单元的数量（回归问题通常为1）

model = SimpleDNN(input_size, hidden_size, output_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练循环
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for inputs_batch, labels_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# 测试模型
model.eval()
with torch.no_grad():
    test_loss = 0.0
    predictions = []

    # 选择测试集的子集以打印预测
    num_samples_to_print = 10
    sample_indices = np.random.choice(len(test_loader.dataset), num_samples_to_print, replace=False)

    for i, (inputs_batch, labels_batch) in enumerate(test_loader):
        outputs = model(inputs_batch)
        test_loss += criterion(outputs, labels_batch).item()

        # 存储选择子集的预测
        if i in sample_indices:
            predictions.append((outputs.numpy(), labels_batch.numpy()))

    average_test_loss = test_loss / len(test_loader)
    print(f'Average Test Loss: {average_test_loss:.4f}')

# 打印选择子集的预测和实际值
for pred, actual in predictions:
    print(f'Prediction: {pred.flatten()}, Actual: {actual.flatten()}')


Epoch [1/3], Loss: 0.0000
Epoch [2/3], Loss: 0.0000
Epoch [3/3], Loss: 0.0000
Average Test Loss: 0.0000
Prediction: [0.13483045 0.77861094 0.8464161  0.73729646 0.58780944 0.4403585
 0.6344154  0.9190117  0.8834994  0.6140028  0.02484199 0.29678282
 0.65279496 0.2899476  0.3656505  0.79785573 0.75623894 0.29198918
 0.9061254  0.09004435 0.6952137  0.12999138 0.482635   0.40310308
 0.8187505  0.08526239 0.6299058  0.20809004 0.45211253 0.6272532
 0.48235294 0.1607351 ], Actual: [0.13649239 0.7732586  0.8552144  0.7423858  0.5910727  0.44388345
 0.6400017  0.930002   0.8810755  0.61865926 0.01976114 0.29666305
 0.6558569  0.29134366 0.36894366 0.79271656 0.75594044 0.2949621
 0.9135992  0.08723605 0.697731   0.12856534 0.4738486  0.40004987
 0.8245925  0.07997212 0.63056076 0.20268957 0.4491081  0.62663746
 0.4793383  0.15517776]


In [2]:
# Print model parameters
for name, param in model.named_parameters():
    print(f'Parameter name: {name}, Size: {param.size()}, Values: {param}')


Parameter name: fc1.weight, Size: torch.Size([8, 3]), Values: Parameter containing:
tensor([[ 0.4414,  0.4792, -0.1353],
        [ 0.6330, -0.0229,  0.1268],
        [-0.3608,  0.2586,  0.4942],
        [-0.3378,  0.5883,  0.1207],
        [ 0.5251,  0.1776,  0.2889],
        [ 0.0062,  0.5336,  0.0980],
        [-0.2695,  0.1472, -0.2660],
        [-0.1478, -0.3153,  0.3683]], requires_grad=True)
Parameter name: fc1.bias, Size: torch.Size([8]), Values: Parameter containing:
tensor([-0.4557, -0.1982, -0.2170, -0.2935,  0.1176, -0.5139,  0.5214, -0.5453],
       requires_grad=True)
Parameter name: fc2.weight, Size: torch.Size([1, 8]), Values: Parameter containing:
tensor([[ 0.2730,  0.0899, -0.1039,  0.2174,  0.0780,  0.3019,  0.0387, -0.1040]],
       requires_grad=True)
Parameter name: fc2.bias, Size: torch.Size([1]), Values: Parameter containing:
tensor([0.1498], requires_grad=True)


In [3]:
# 提取模型参数
weights_fc1 = model.fc1.weight.data.numpy().flatten() * 2**16
bias_fc1 = model.fc1.bias.data.numpy().flatten() * 2**16
weights_fc2 = model.fc2.weight.data.numpy().flatten() * 2**16
bias_fc2 = model.fc2.bias.data.numpy().flatten() * 2**16

# 将参数转换为整数并四舍五入
weights_fc1_int = weights_fc1.round().astype(int)
bias_fc1_int = bias_fc1.round().astype(int)
weights_fc2_int = weights_fc2.round().astype(int)
bias_fc2_int = bias_fc2.round().astype(int)

# 定义一个函数将浮点数转换为二进制字符串
def float_to_bin(num):
    # 处理负数的补码表示
    if num < 0:
        num =  num + 2**32
    # 转换为16位二进制字符串
    bin_str = bin(num)[2:].zfill(32)
    return bin_str

# 打印模型参数的二进制表示
print("weights_fc1:")
for weight in weights_fc1_int:
    print(f"{float_to_bin(weight)}\t{weight}")

print("\nbias_fc1:")
for bias in bias_fc1_int:
    print(f"{float_to_bin(bias)}\t{bias}")

print("\nweights_fc2:")
for weight in weights_fc2_int:
    print(f"{float_to_bin(weight)}\t{weight}")

print("\nbias_fc2:")
for bias in bias_fc2_int:
    print(f"{float_to_bin(bias)}\t{bias}")


weights_fc1:
00000000000000000111000100000000	28928
00000000000000000111101010101101	31405
11111111111111111101110101100000	-8864
00000000000000001010001000001101	41485
11111111111111111111101000100011	-1501
00000000000000000010000001110101	8309
11111111111111111010001110100010	-23646
00000000000000000100001000110100	16948
00000000000000000111111010000111	32391
11111111111111111010100110000100	-22140
00000000000000001001011010011011	38555
00000000000000000001111011101001	7913
00000000000000001000011001101110	34414
00000000000000000010110101111010	11642
00000000000000000100100111110100	18932
00000000000000000000000110011000	408
00000000000000001000100010011010	34970
00000000000000000001100100010111	6423
11111111111111111011101100000000	-17664
00000000000000000010010110101101	9645
11111111111111111011101111100111	-17433
11111111111111111101101000101010	-9686
11111111111111111010111101000110	-20666
00000000000000000101111001001000	24136

bias_fc1:
11111111111111111000101101010100	-29868
1

In [6]:
def relu(x):
    return np.max(x, 0)
w1 = np.array([[ 0.4414,  0.4792, -0.1353],
        [ 0.6330, -0.0229,  0.1268],
        [-0.3608,  0.2586,  0.4942],
        [-0.3378,  0.5883,  0.1207],
        [ 0.5251,  0.1776,  0.2889],
        [ 0.0062,  0.5336,  0.0980],
        [-0.2695,  0.1472, -0.2660],
        [-0.1478, -0.3153,  0.3683]])
w2 = np.array([0.2730,  0.0899, -0.1039,  0.2174,  0.0780,  0.3019,  0.0387, -0.1040])

b1 = np.array([-0.4557, -0.1982, -0.2170, -0.2935,  0.1176, -0.5139,  0.5214, -0.5453])
b2 = np.array([0.1498])

label = np.array([0.1498])
data_in = np.array([4.205534458467607,1.999356317214453,26])


def relu(x):
    return np.maximum(x, 0)

# 输入

# 第一层计算
layer1_output = relu(np.dot(data_in, w1.T) + b1)

# 第二层计算
output = np.dot(layer1_output, w2) + b2
print(np.dot(data_in, w1.T) + b1)
# 输出
print("输出:", output)


[-1.15908554  5.71491805 11.63187671  2.60029178 10.19241183  3.12703084
 -7.23368629  7.77852496]
输出: [0.95041471]
